Вот подробный построчный разбор кода файла **app-plotted.ipynb**:



In [ ]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.animation import FuncAnimation, FFMpegWriter
import tempfile
import os
from numba import jit

**Импорт библиотек:**  
- gradio — для создания веб-интерфейса  
- numpy — для работы с массивами  
- matplotlib — для построения графиков и анимаций  
- tempfile, os — для временных файлов  
- numba — для ускорения вычислений с помощью JIT-компиляции

---



In [ ]:
# --- Константы и цвета ---
NEIGHBOURHOOD = ((-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1))
EMPTY, TREE, FIRE = 0, 1, 2
colors_list = [(0.2,0,0), (0,0.5,0), (1,0,0), 'orange']
cmap = colors.ListedColormap(colors_list)
bounds = [0,1,2,3,4]
norm = colors.BoundaryNorm(bounds, cmap.N)

**Константы:**  
- NEIGHBOURHOOD — относительные координаты соседей (8 направлений)
- EMPTY, TREE, FIRE — коды состояний клетки
- colors_list — цвета для визуализации состояний
- cmap, bounds, norm — настройки colormap для matplotlib

---



In [ ]:
@jit(nopython=True)
def iterate_optimized(X_curr, p_growth, f_lightning, ny_dim, nx_dim):
    X_next = np.zeros((ny_dim, nx_dim), dtype=X_curr.dtype)
    X_values = np.zeros(3, dtype=np.int64)  # [EMPTY, TREE, FIRE]
    for r in range(1, ny_dim - 1):
        for c in range(1, nx_dim - 1):
            current_state = X_curr[r, c]
            X_values[current_state] += 1  # Считаем количество клеток каждого типа
            if current_state == EMPTY:
                if np.random.random() <= p_growth:
                    X_next[r, c] = TREE
            elif current_state == TREE:
                X_next[r, c] = TREE
                caught_fire_from_neighbor = False
                for dr, dc in NEIGHBOURHOOD:
                    if X_curr[r + dr, c + dc] == FIRE:
                        X_next[r, c] = FIRE
                        caught_fire_from_neighbor = True
                        break
                if not caught_fire_from_neighbor:
                    if np.random.random() <= f_lightning:
                        X_next[r, c] = FIRE
    return X_next, X_values

**Функция одного шага клеточного автомата:**  
- Использует JIT для ускорения  
- Для каждой клетки считает новое состояние  
- Пустая клетка может вырасти в дерево с вероятностью p_growth  
- Дерево может загореться от соседа или от молнии  
- Возвращает новое состояние поля и статистику по количеству клеток каждого типа

---



In [ ]:
def create_forest_animation(seed=0, frames=100, size=200, p=0.02, f=0.0001, forest_fraction=0.2, fps=20):
    np.random.seed(seed)
    ny, nx = size, size
    X = np.zeros((ny, nx), dtype=np.int8)
    interior_mask = np.random.random(size=(ny-2, nx-2)) < forest_fraction
    X[1:ny-1, 1:nx-1] = np.where(interior_mask, TREE, EMPTY).astype(np.int8)

    stats = np.zeros((frames, 3), dtype=np.int64)  # [frame, state]
    dpi = 100
    figsize = (size / dpi, size / dpi)
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_axis_off()
    plt.tight_layout(pad=0)
    im = ax.imshow(X, cmap=cmap, norm=norm, interpolation='nearest')

    def animate(i):
        nonlocal X
        X, X_values = iterate_optimized(X, p, f, ny, nx)
        stats[i, :] = X_values
        im.set_data(X)
        return [im]

    writer = FFMpegWriter(fps=fps, bitrate=int(size*size*fps*0.08))
    tmp_file = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False)
    ani = FuncAnimation(fig, animate, frames=frames, interval=1000/fps, blit=True)
    ani.save(tmp_file.name, writer=writer)
    plt.close(fig)
    return tmp_file.name, stats

**Генерация анимации:**  
- Инициализация поля леса с заданной долей деревьев  
- Для каждого кадра вызывает iterate_optimized, собирает статистику  
- Сохраняет анимацию в mp4-файл  
- Возвращает путь к видео и статистику по кадрам

---



In [ ]:
def gradio_grid(seed, grid_size, size, frames, p, f, forest_fraction):
    video_items = []
    for i in range(grid_size):
        for j in range(grid_size):
            cell_seed = seed + i * grid_size + j
            video_path, stats = create_forest_animation(
                seed=cell_seed, frames=frames, size=size, p=p, f=f, forest_fraction=forest_fraction
            )
            label = f"Seed: {cell_seed} ({i},{j})"
            # Строим график для всех типов клеток
            plt.figure(figsize=(5,3))
            x = np.arange(frames)
            plt.plot(x, stats[:, 0], color='gray', label='EMPTY')
            plt.plot(x, stats[:, 1], color='green', label='TREE')
            plt.plot(x, stats[:, 2], color='red', label='FIRE')
            # Средние значения
            mean_empty = stats[:, 0].mean()
            mean_tree = stats[:, 1].mean()
            mean_fire = stats[:, 2].mean()
            plt.axhline(mean_empty, color='gray', linestyle='--', alpha=0.7, label=f'EMPTY mean: {mean_empty:.0f}')
            plt.axhline(mean_tree, color='green', linestyle='--', alpha=0.7, label=f'TREE mean: {mean_tree:.0f}')
            plt.axhline(mean_fire, color='red', linestyle='--', alpha=0.7, label=f'FIRE mean: {mean_fire:.0f}')
            plt.xlabel('Frame')
            plt.ylabel('Cell count')
            plt.title(f'Forest size: {size} | Frames: {frames} | Seed: {cell_seed} | $p_f=$ {mean_empty*p:.2f}', fontsize=10)
            plt.legend()
            plt.tight_layout()
            img_tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
            plt.savefig(img_tmp.name)
            plt.close()
            # Добавляем видео и график в gallery
            video_items.append((video_path, label))
            video_items.append((img_tmp.name, f"Cells vs Time: {label}"))
    return video_items

**Генерация сетки анимаций и графиков:**  
- Для каждой ячейки сетки создаёт свою анимацию с уникальным seed  
- Строит график изменения числа клеток каждого типа по времени  
- Добавляет горизонтальные линии средних значений  
- Сохраняет график во временный PNG-файл  
- Возвращает список пар (видео, график) для отображения в Gradio

---



In [ ]:
gr.Interface(
    fn=gradio_grid,
    inputs=[
        gr.Number(label="Random Seed", value=0),
        gr.Slider(1, 10, value=1, step=1, label="Grid Size GR (GRxGR)"),
        gr.Slider(50, 2000, value=100, step=10, label="Forest Size (NxN)"),
        gr.Slider(10, 20000, value=50, step=1, label="Number of Frames"),
        gr.Slider(0.001, 0.1, value=0.02, step=0.001, label="Tree Growth Probability (p)"),
        gr.Slider(0.0, 0.01, value=0.0001, step=0.0001, label="Lightning Probability (f)"),
        gr.Slider(0.01, 1.0, value=0.2, step=0.01, label="Initial Forest Fraction"),
    ],
    outputs=gr.Gallery(label="Forest Fire Animation Grid (MP4 + TREE Graph)", columns=2, height="auto"),
    title="Forest Fire Cellular Automaton Animation Grid",
    description="Simulates a GRxGR grid of independent forest fire cellular automata. For each cell, shows MP4 animation and TREE count graph."
).launch()

**Gradio-интерфейс:**  
- Позволяет пользователю задавать параметры симуляции  
- Выводит сетку анимаций и графиков  
- Каждый элемент — пара: видео и график для одной симуляции

---

**Итого:**  
Этот код моделирует распространение лесного пожара на клеточном автомате, строит анимацию и графики по каждому запуску, и предоставляет удобный веб-интерфейс для экспериментов с параметрами.

Similar code found with 1 license type

---------------

------------

In [ ]:
import gradio as gr  # Импортируем Gradio для создания веб-интерфейса
import numpy as np  # Импортируем numpy для работы с массивами
import matplotlib.pyplot as plt  # Импортируем matplotlib для построения графиков и анимаций
from matplotlib import colors  # Импортируем модуль цветов для визуализации состояний клеток
from matplotlib.animation import FuncAnimation, FFMpegWriter  # Для создания и сохранения анимаций
import tempfile  # Для создания временных файлов
import os  # Для работы с файловой системой
from numba import jit  # Для ускорения вычислений с помощью JIT-компиляции

# --- Константы и цвета ---
NEIGHBOURHOOD = ((-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1))  # Смещения для соседей (8 направлений)
EMPTY, TREE, FIRE = 0, 1, 2  # Коды состояний клетки: пусто, дерево, огонь
colors_list = [(0.2,0,0), (0,0.5,0), (1,0,0), 'orange']  # Цвета для визуализации состояний
cmap = colors.ListedColormap(colors_list)  # Создаём colormap для matplotlib
bounds = [0,1,2,3,4]  # Границы для colormap
norm = colors.BoundaryNorm(bounds, cmap.N)  # Нормализация цветов

@jit(nopython=True)  # Ускоряем функцию с помощью numba JIT
def iterate_optimized(X_curr, p_growth, f_lightning, ny_dim, nx_dim):
    X_next = np.zeros((ny_dim, nx_dim), dtype=X_curr.dtype)  # Новое состояние поля
    X_values = np.zeros(3, dtype=np.int64)  # Счётчики для каждого состояния
    for r in range(1, ny_dim - 1):  # Проходим по внутренним клеткам (без границ)
        for c in range(1, nx_dim - 1):
            current_state = X_curr[r, c]  # Текущее состояние клетки
            X_values[current_state] += 1  # Увеличиваем счётчик для этого состояния
            if current_state == EMPTY:  # Если клетка пуста
                if np.random.random() <= p_growth:  # Может вырасти дерево с вероятностью p_growth
                    X_next[r, c] = TREE
            elif current_state == TREE:  # Если клетка дерево
                X_next[r, c] = TREE  # По умолчанию остаётся деревом
                caught_fire_from_neighbor = False
                for dr, dc in NEIGHBOURHOOD:  # Проверяем всех соседей
                    if X_curr[r + dr, c + dc] == FIRE:  # Если сосед горит
                        X_next[r, c] = FIRE  # Дерево загорается
                        caught_fire_from_neighbor = True
                        break
                if not caught_fire_from_neighbor:  # Если не загорелось от соседа
                    if np.random.random() <= f_lightning:  # Может загореться от молнии
                        X_next[r, c] = FIRE
    return X_next, X_values  # Возвращаем новое поле и статистику

def create_forest_animation(seed=0, frames=100, size=200, p=0.02, f=0.0001, forest_fraction=0.2, fps=20):
    np.random.seed(seed)  # Фиксируем seed для воспроизводимости
    ny, nx = size, size  # Размеры поля
    X = np.zeros((ny, nx), dtype=np.int8)  # Инициализируем поле пустыми клетками
    interior_mask = np.random.random(size=(ny-2, nx-2)) < forest_fraction  # Маска для начального заполнения деревьями
    X[1:ny-1, 1:nx-1] = np.where(interior_mask, TREE, EMPTY).astype(np.int8)  # Заполняем внутреннюю часть поля

    stats = np.zeros((frames, 3), dtype=np.int64)  # Массив для хранения статистики по кадрам
    dpi = 100  # Разрешение для matplotlib
    figsize = (size / dpi, size / dpi)  # Размер фигуры
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)  # Создаём фигуру и ось
    ax.set_xticks([])  # Убираем оси
    ax.set_yticks([])
    ax.set_axis_off()
    plt.tight_layout(pad=0)
    im = ax.imshow(X, cmap=cmap, norm=norm, interpolation='nearest')  # Показываем начальное состояние

    def animate(i):  # Функция для обновления каждого кадра
        nonlocal X
        X, X_values = iterate_optimized(X, p, f, ny, nx)  # Выполняем один шаг симуляции
        stats[i, :] = X_values  # Сохраняем статистику
        im.set_data(X)  # Обновляем изображение
        #print(stats[i].sum())
        return [im]

    writer = FFMpegWriter(fps=fps, bitrate=int(size*size*fps*0.08))  # Настраиваем writer для mp4
    tmp_file = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False)  # Временный файл для видео
    ani = FuncAnimation(fig, animate, frames=frames, interval=1000/fps, blit=True)  # Создаём анимацию
    ani.save(tmp_file.name, writer=writer)  # Сохраняем анимацию в файл
    plt.close(fig)  # Закрываем фигуру
    return tmp_file.name, stats  # Возвращаем путь к видео и статистику

def gradio_grid(seed, grid_size, size, frames, p, f, forest_fraction):
    video_items = []  # Список для хранения видео и графиков
    for i in range(grid_size):  # Проходим по строкам сетки
        for j in range(grid_size):  # Проходим по столбцам сетки
            cell_seed = seed + i * grid_size + j  # Уникальный seed для каждой ячейки
            video_path, stats = create_forest_animation(
                seed=cell_seed, frames=frames, size=size, p=p, f=f, forest_fraction=forest_fraction
            )
            label = f"Seed: {cell_seed} ({i},{j})"  # Подпись для видео
            # Строим график для всех типов клеток
            plt.figure(figsize=(5,3))
            x = np.arange(frames)
            plt.plot(x, stats[:, 0], color='gray', label='EMPTY')  # График пустых клеток
            plt.plot(x, stats[:, 1], color='green', label='TREE')  # График деревьев
            plt.plot(x, stats[:, 2], color='red', label='FIRE')  # График огня
            # Средние значения
            mean_empty = stats[:, 0].mean()
            mean_tree = stats[:, 1].mean()
            mean_fire = stats[:, 2].mean()
            plt.axhline(mean_empty, color='gray', linestyle='--', alpha=0.7, label=f'EMPTY mean: {mean_empty:.0f}')
            plt.axhline(mean_tree, color='green', linestyle='--', alpha=0.7, label=f'TREE mean: {mean_tree:.0f}')
            plt.axhline(mean_fire, color='red', linestyle='--', alpha=0.7, label=f'FIRE mean: {mean_fire:.0f}')
            plt.xlabel('Frame')
            plt.ylabel('Cell count')
            plt.title(f'Forest size: {size} | Frames: {frames} | Seed: {cell_seed} | $p_f=$ {mean_empty*p:.2f}', fontsize=10)
            plt.legend()
            plt.tight_layout()
            img_tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)  # Временный файл для графика
            plt.savefig(img_tmp.name)  # Сохраняем график
            plt.close()
            # Добавляем видео и график в gallery
            video_items.append((video_path, label))  # Добавляем видео
            video_items.append((img_tmp.name, f"Cells vs Time: {label}"))  # Добавляем график
    return video_items  # Возвращаем список для галереи

# Создаём Gradio-интерфейс
gr.Interface(
    fn=gradio_grid,  # Основная функция для запуска
    inputs=[
        gr.Number(label="Random Seed", value=0),  # Поле для ввода seed
        gr.Slider(1, 10, value=1, step=1, label="Grid Size GR (GRxGR)"),  # Размер сетки
        gr.Slider(50, 2000, value=100, step=10, label="Forest Size (NxN)"),  # Размер леса
        gr.Slider(10, 20000, value=50, step=1, label="Number of Frames"),  # Количество кадров
        gr.Slider(0.001, 0.1, value=0.02, step=0.001, label="Tree Growth Probability (p)"),  # Вероятность роста дерева
        gr.Slider(0.0, 0.01, value=0.0001, step=0.0001, label="Lightning Probability (f)"),  # Вероятность молнии
        gr.Slider(0.01, 1.0, value=0.2, step=0.01, label="Initial Forest Fraction"),  # Начальная доля деревьев
    ],
    outputs=gr.Gallery(label="Forest Fire Animation Grid (MP4 + TREE Graph)", columns=2, height="auto"),  # Галерея для вывода результатов
    title="Forest Fire Cellular Automaton Animation Grid",  # Заголовок интерфейса
    description="Simulates a GRxGR grid of independent forest fire cellular automata. For each cell, shows MP4 animation and TREE count graph."  # Описание
).launch()  # Запуск интерфейса

----

In [1]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.animation import FuncAnimation, FFMpegWriter
import tempfile
import os
from numba import jit

# --- Константы и цвета ---
NEIGHBOURHOOD = ((-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1))
EMPTY, TREE, FIRE = 0, 1, 2
colors_list = [(0.2,0,0), (0,0.5,0), (1,0,0), 'orange']
cmap = colors.ListedColormap(colors_list)
bounds = [0,1,2,3,4]
norm = colors.BoundaryNorm(bounds, cmap.N)

@jit(nopython=True)
def iterate_optimized(X_curr, p_growth, f_lightning, ny_dim, nx_dim):
    X_next = np.zeros((ny_dim, nx_dim), dtype=X_curr.dtype)
    X_values = np.zeros(3, dtype=np.int64)  # [EMPTY, TREE, FIRE]
    for r in range(1, ny_dim - 1):
        for c in range(1, nx_dim - 1):
            current_state = X_curr[r, c]
            X_values[current_state] += 1  # Считаем количество клеток каждого типа
            if current_state == EMPTY:
                if np.random.random() <= p_growth:
                    X_next[r, c] = TREE
            elif current_state == TREE:
                X_next[r, c] = TREE
                caught_fire_from_neighbor = False
                for dr, dc in NEIGHBOURHOOD:
                    if X_curr[r + dr, c + dc] == FIRE:
                        X_next[r, c] = FIRE
                        caught_fire_from_neighbor = True
                        break
                if not caught_fire_from_neighbor:
                    if np.random.random() <= f_lightning:
                        X_next[r, c] = FIRE
    return X_next, X_values

def create_forest_animation(seed=0, frames=100, size=200, p=0.02, f=0.0001, forest_fraction=0.2, fps=20):
    np.random.seed(seed)
    ny, nx = size, size
    X = np.zeros((ny, nx), dtype=np.int8)
    interior_mask = np.random.random(size=(ny-2, nx-2)) < forest_fraction
    X[1:ny-1, 1:nx-1] = np.where(interior_mask, TREE, EMPTY).astype(np.int8)

    stats = np.zeros((frames, 3), dtype=np.int64)  # [frame, state]
    dpi = 100
    figsize = (size / dpi, size / dpi)
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_axis_off()
    plt.tight_layout(pad=0)
    im = ax.imshow(X, cmap=cmap, norm=norm, interpolation='nearest')

    def animate(i):
        nonlocal X
        X, X_values = iterate_optimized(X, p, f, ny, nx)
        stats[i, :] = X_values
        im.set_data(X)
        #print(stats[i].sum())
        return [im]

    writer = FFMpegWriter(fps=fps, bitrate=int(size*size*fps*0.08))
    tmp_file = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False)
    ani = FuncAnimation(fig, animate, frames=frames, interval=1000/fps, blit=True)
    ani.save(tmp_file.name, writer=writer)
    plt.close(fig)
    return tmp_file.name, stats

def gradio_grid(seed, grid_size, size, frames, p, f, forest_fraction):
    video_items = []
    for i in range(grid_size):
        for j in range(grid_size):
            cell_seed = seed + i * grid_size + j
            video_path, stats = create_forest_animation(
                seed=cell_seed, frames=frames, size=size, p=p, f=f, forest_fraction=forest_fraction
            )
            label = f"Seed: {cell_seed} ({i},{j})"
            # Строим график для всех типов клеток
            plt.figure(figsize=(5,3))
            x = np.arange(frames)
            plt.plot(x, stats[:, 0], color='gray', label='EMPTY')
            plt.plot(x, stats[:, 1], color='green', label='TREE')
            plt.plot(x, stats[:, 2], color='red', label='FIRE')
            # Средние значения
            mean_empty = stats[:, 0].mean()
            mean_tree = stats[:, 1].mean()
            mean_fire = stats[:, 2].mean()
            plt.axhline(mean_empty, color='gray', linestyle='--', alpha=0.7, label=f'EMPTY mean: {mean_empty:.0f}')
            plt.axhline(mean_tree, color='green', linestyle='--', alpha=0.7, label=f'TREE mean: {mean_tree:.0f}')
            plt.axhline(mean_fire, color='red', linestyle='--', alpha=0.7, label=f'FIRE mean: {mean_fire:.0f}')
            plt.xlabel('Frame')
            plt.ylabel('Cell count')
            plt.title(f'Forest size: {size} | Frames: {frames} | Seed: {cell_seed} | $p_f=$ {mean_empty*p:.2f}', fontsize=10)
            plt.legend()
            plt.tight_layout()
            img_tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
            plt.savefig(img_tmp.name)
            plt.close()
            # Добавляем видео и график в gallery
            video_items.append((video_path, label))
            video_items.append((img_tmp.name, f"Cells vs Time: {label}"))
    return video_items
gr.Interface(
    fn=gradio_grid,
    inputs=[
        gr.Number(label="Random Seed", value=0),
        gr.Slider(1, 10, value=1, step=1, label="Grid Size GR (GRxGR)"),
        gr.Slider(50, 2000, value=100, step=10, label="Forest Size (NxN)"),
        gr.Slider(10, 20000, value=50, step=1, label="Number of Frames"),
        gr.Slider(0.001, 0.1, value=0.02, step=0.001, label="Tree Growth Probability (p)"),
        gr.Slider(0.0, 0.01, value=0.0001, step=0.0001, label="Lightning Probability (f)"),
        gr.Slider(0.01, 1.0, value=0.2, step=0.01, label="Initial Forest Fraction"),
    ],
    outputs=gr.Gallery(label="Forest Fire Animation Grid (MP4 + TREE Graph)", columns=2, height="auto"),
    title="Forest Fire Cellular Automaton Animation Grid",
    description="Simulates a GRxGR grid of independent forest fire cellular automata. For each cell, shows MP4 animation and TREE count graph."
).launch()

* Running on local URL:  http://127.0.0.1:7912

To create a public link, set `share=True` in `launch()`.
